In [1]:
import torch
from torchvision.transforms import v2


In [2]:
from torchvision.models import shufflenet_v2_x0_5

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
import numpy as np
from numpy import asarray

In [5]:
from PIL import Image

In [6]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [7]:
import pathlib

In [8]:
dataset_url='/content/drive/MyDrive/plant_leaf_dataset/dataset2/'

In [9]:
dataset_dir=pathlib.Path(dataset_url)

In [10]:
dataset_dict={
    'Mango': list(dataset_dir.glob('Mango (P0)/*.JPG')),
    'Arjun': list(dataset_dir.glob('Arjun (P1)/*.JPG')),
    'Alstonia_Scholaris': list(dataset_dir.glob('Alstonia Scholaris (P2)/*.JPG')),
    'Gauva': list(dataset_dir.glob('Gauva (P3)/*.JPG')),
    'Jamun': list(dataset_dir.glob('Jamun (P5)/*.JPG')),
    'Jatropha':list(dataset_dir.glob('Jatropha (P6)/*.JPG')),
    'Pongamia_Pinnata':list(dataset_dir.glob('Pongamia Pinnata (P7)/*.JPG')),
    'Basil':list(dataset_dir.glob('Basil (P8)/*.JPG')),
    'Pomegranate':list(dataset_dir.glob('Pomegranate (P9)/*.JPG')),
    'Lemon':list(dataset_dir.glob('Lemon (P10)/*.JPG')),
    'Chinar':list(dataset_dir.glob('Chinar (P11)/*.JPG')),
}

In [11]:
print(dataset_dict)

{'Mango': [PosixPath('/content/drive/MyDrive/plant_leaf_dataset/dataset2/Mango (P0)/0001_0011.JPG'), PosixPath('/content/drive/MyDrive/plant_leaf_dataset/dataset2/Mango (P0)/0001_0008.JPG'), PosixPath('/content/drive/MyDrive/plant_leaf_dataset/dataset2/Mango (P0)/0001_0014.JPG'), PosixPath('/content/drive/MyDrive/plant_leaf_dataset/dataset2/Mango (P0)/0001_0009.JPG'), PosixPath('/content/drive/MyDrive/plant_leaf_dataset/dataset2/Mango (P0)/0001_0010.JPG'), PosixPath('/content/drive/MyDrive/plant_leaf_dataset/dataset2/Mango (P0)/0001_0017.JPG'), PosixPath('/content/drive/MyDrive/plant_leaf_dataset/dataset2/Mango (P0)/0001_0018.JPG'), PosixPath('/content/drive/MyDrive/plant_leaf_dataset/dataset2/Mango (P0)/0001_0006.JPG'), PosixPath('/content/drive/MyDrive/plant_leaf_dataset/dataset2/Mango (P0)/0001_0021.JPG'), PosixPath('/content/drive/MyDrive/plant_leaf_dataset/dataset2/Mango (P0)/0001_0007.JPG'), PosixPath('/content/drive/MyDrive/plant_leaf_dataset/dataset2/Mango (P0)/0001_0003.JPG'),

In [12]:
class_label_dict={
    'Mango':0,
    'Arjun':1,
    'Alstonia_Scholaris':2,
    'Gauva':3,
    'Jamun':4,
    'Jatropha':5,
    'Pongamia_Pinnata':6,
    'Basil':7,
    'Pomegranate':8,
    'Lemon':9,
    'Chinar':10,
}
print(class_label_dict['Chinar'])

10


Affine transformation to the images n+1 dimensions

In [13]:
affine_image = v2.Compose([
    v2.RandomAffine(degrees=(30, 70), translate=(0.1, 0.3), scale=(0.5, 0.75)),
])


Random Zoom

In [14]:
random_zoom=v2.Compose([
    v2.RandomResizedCrop(size=(224, 224))
])

180 degree rotation

In [15]:
rotation=v2.Compose([
    v2.RandomRotation(degrees=(0, 180))
])

Resizing the images to 224 X 224 and augmenting images

In [16]:
X=[]
y=[]
count=0
for key in dataset_dict.keys():
  for values in dataset_dict[key]:
        resized_image=Image.open(values).resize((224,224))
        X.append(resized_image)
        y.append(class_label_dict[key])
        affined_image=affine_image(resized_image)
        X.append(affined_image)
        y.append(class_label_dict[key])
        zoomed_image=random_zoom(resized_image)
        X.append(zoomed_image)
        y.append(class_label_dict[key])
        rotated_image=rotation(resized_image)
        X.append(rotated_image)
        y.append(class_label_dict[key])
        count+=1
        # if count>4:
        #   break
  # break

In [17]:
print(len(X))
print(len(y))

9108
9108


In [18]:
for image in X:
  image=np.asarray(image)/255

In [19]:
(X_train,X_test,y_train,y_test)=train_test_split(X,y,test_size=0.2,random_state=0)

In [20]:
print(len(X_train),len(y_train))

7286 7286


In [21]:
print(type(y[0]))

<class 'int'>


In [ ]:
model=shufflenet_v2_x0_5()

In [ ]:
X_train_array=[]
for _ in X_train:
  arr=asarray(_,dtype=np.float32)/255.0
  X_train_array.append(arr)
X_train_tensor=torch.stack([torch.from_numpy(array) for array in X_train_array])


# X_train_tensor=torch.tensor(X_tensor_list)
# print(X_train_tensor.size)
# X_train_tensor=torch.from_numpy(X_train)
y_train_tensor=torch.tensor(y_train)

In [ ]:
X_train_tensor = X_train_tensor.permute(0, 3, 1, 2)

In [ ]:
output=model(X_train_tensor)
classification_class=torch.argmax(output, dim=1)
print(classification_class)

tensor([788,  41, 189,  41, 788, 189, 189, 189, 189, 189, 189, 189, 788, 189,
        189, 189])


# Implementing ShuffleNet CNN on the pre processed dataset 2

In [22]:
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from torch.utils.data.dataset import random_split

In [23]:
# Define global variables:
TOTAL_CLASSES = 11
NUM_EPOCHS = 10
batch_size = 32

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Load pre-trained googleNet Model
# model = models.googlenet(pretrained=True)

In [ ]:
#Loading shufflenet
# model = torch.hub.load('pytorch/vision:v0.10.0', 'shufflenet_v2_x1_0', pretrained=True)


Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
# Shufflenet defined with 1000 different classes. Need to change it with the size of our class number.
# num_classes = TOTAL_CLASSES #len(classification_class)
# in_features = model.fc.in_features
# model.fc = nn.Linear(in_features, num_classes)


In [24]:
# Define loss function
criterion = nn.CrossEntropyLoss()

NameError: name 'model' is not defined

In [25]:
#Making the dataset ready for taining out of x_train and y_train

class CustomDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)
            # input_batch = image.unsqueeze(0)

        return image, label

# Define your transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Create your Dataset
dataset = CustomDataset(images=X_train, labels=y_train, transform=transform)
print(dataset)



In [ ]:
# print(dataset(0))

TypeError: 'CustomDataset' object is not callable

In [26]:


model = torch.hub.load('pytorch/vision:v0.10.0', 'shufflenet_v2_x1_0', pretrained=False)
# model = models.shufflenet_v2_x1_0(pretrained=False)

# model = Model()
# device = ('cuda' if torch.cuda.is_available() else 'cpu')

model = model.to(device)
model = torch.nn.DataParallel(model)
model.fc = nn.Linear(1024, 11)

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [27]:
# Optimize
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [28]:
# Once the dataset is ready split the x_train, y_train dataset into training and validation dataset
# 80% for training 20% for validation
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, validation_dataset = random_split(dataset, [train_size, val_size])
print(len(train_dataset))
print(len(validation_dataset))

5828
1458


In [29]:
# Create data loader to feed the CNN
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)


In [30]:
# Iterate over the dataset
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# input_tensor = input_tensor.to('cuda')
# tensor_on_gpu = tensor_on_gpu.to(device)

# Now both tensors are on the same device, so the operation should work
# result = torch.some_function(tensor_on_gpu, tensor_on_cpu)
for epoch in range(NUM_EPOCHS):
    print('--------------------')
    print(f'EPOCH: {epoch + 1}/{NUM_EPOCHS}')
    model.train()
    batch = 0
    for images, labels in train_loader:
        # Forward pass
        batch = batch+1
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batch%10 == 0:
          print(f'Loss for {batch}/{len(train_loader)} of {epoch} epoch is: ', loss)


--------------------
EPOCH: 1/10
Loss for 10/183 of 0 epoch is:  tensor(6.2779, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss for 20/183 of 0 epoch is:  tensor(5.3119, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss for 30/183 of 0 epoch is:  tensor(3.9392, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss for 40/183 of 0 epoch is:  tensor(3.2897, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss for 50/183 of 0 epoch is:  tensor(2.8242, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss for 60/183 of 0 epoch is:  tensor(2.5374, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss for 70/183 of 0 epoch is:  tensor(2.6406, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss for 80/183 of 0 epoch is:  tensor(2.2954, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss for 90/183 of 0 epoch is:  tensor(2.1823, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss for 100/183 of 0 epoch is:  tensor(2.4270, device='cuda:0', grad_fn=<NllLossBackward0>)
Loss for 110/183 of 0 epoch is:  tensor(2.2187

In [33]:
#Evaluate the model's output
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in validation_loader:

        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    accuracy = correct / total
    # print('Accuracy on validation set: {}%'.format(100 * accuracy))
    print(f'Accuracy {accuracy*100:.2f}%')


Accuracy 27.71%
